In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
import shutil

In [3]:
data_dir = os.getcwd()
img_dir = os.path.join(data_dir, 'images/')
excel_path = os.path.join(data_dir, 'hh_video_data.xlsx')
label_path = os.path.join(data_dir, 'handhygiene_label.csv') 
df_ex = pd.read_excel(excel_path)

In [4]:
def move_target_data(img_dir, target_str):
    nonetarget_dir = os.path.join(img_dir, target_str)
    rest_img_dir = os.path.join(img_dir, '*.jpg')
    !mv $rest_img_dir $nonetarget_dir

In [5]:
def add_to_data_dict(video_id, img_date, img_name, clip_num):
    hh_dict['video_id'].append(video_id)
    hh_dict['date'].append(img_date)
    hh_dict['img_name'].append(img_name+'.jpg')
    hh_dict['clip_num'].append(clip_num)
    return 

In [59]:
hh_dict = {'video_id':[], 'date':[], 'img_name':[], 'clip_num':[]}
#target_str = 'clean'
target_dir = os.path.join(img_dir, 'clean')

def label_data(img_dir):
    img_total = glob(os.path.join(img_dir, '*.jpg'))
    img_total.sort()
    cnt = 0
    clip_num = 0

    for image in tqdm(img_total):
        img_name= image.split('/')[-1].split('.')[0]
        video_id = int(img_name.split('_')[0])
        img_date = img_name.split('_')[1]
        img_frame_num = img_name[-6:] # ex: 000001

        for vid, target in df_ex[['video_id', 'target_frame']].values:
            
            if video_id is not vid:
                continue
            if target is np.nan:
                continue
            
            #target_str = 'clean'
            targets = {'clean': target}
            clips = targets['clean'].strip().split(',')
            
            for n, el in enumerate(clips):
                start = el.strip().split('-')[0]
                end = el.strip().split('-')[1]
                
                # if current frame is in range of (start, end)
                if int(img_frame_num) in range(int(start), int(end)+1):
                    shutil.move(image, target_dir) # move data to target directory
                    add_to_data_dict(video_id, img_date, img_name, clip_num)
                    
                    if int(img_frame_num) == int(end):
                        clip_num += 1
                        
                    cnt += 1
                    break
                    

    print("{} images moved to {}".format(cnt, target_dir))
    move_target_data(img_dir, 'notclean')
    
    return

In [60]:
label_data(img_dir)

100%|██████████| 3402/3402 [00:01<00:00, 1894.35it/s]


1031 images moved to /data/handhygiene/images/clean


In [62]:
df = pd.DataFrame(hh_dict)
pd.DataFrame.to_csv(df, label_path, index=False)